## Install Dependencies

1. `pip install qwen-tts`
2. `pip uninstall -y torch torchaudio torchvision`
3. `conda install pytorch torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia -y`


In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["PYTORCH_SDP_ENABLED"] = "0"
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "0"
os.environ["TORCH_USE_CUDA_DSA"] = "0"

In [ ]:
import torch

torch.backends.cuda.enable_flash_sdp(False)
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_math_sdp(True)


print("Torch CUDA runtime:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))
print("Capability:", torch.cuda.get_device_capability(0))

Torch CUDA runtime: 12.1
CUDA available: True
GPU: NVIDIA GeForce GTX 1660 Ti
Capability: (7, 5)


### Install SOX

#### Step 1️⃣ Download SoX (official source)

Go here (official, no tricks):

👉 [http://sox.sourceforge.net/](http://sox.sourceforge.net/)

Click **“Download SoX for Windows”**
Get the **Windows binary installer** (usually `sox-14.4.x-win32.exe` or similar).

---

#### Step 2️⃣ Install it (IMPORTANT options)

During installation:

- ✅ **Check “Add SoX to PATH”**
- Leave everything else default
- Finish install

This PATH checkbox is the most important part.

---

#### Step 3️⃣ Verify SoX works (outside Python)

Open a **new terminal** (important), then run:

```bat
sox --version
```

Expected output (example):

```
sox: SoX v14.4.2
```

If you see that → SoX is installed correctly.


In [3]:
import torch
from qwen_tts import Qwen3TTSModel
import gc

# -----------------------------
# Environment info
# -----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

print("===== ENVIRONMENT =====")
print("Device selected :", device)
print("Torch version   :", torch.__version__)
print("CUDA available  :", torch.cuda.is_available())

if device == "cuda":
    print("CUDA version    :", torch.version.cuda)
    print("GPU name        :", torch.cuda.get_device_name(0))
    print(
        "GPU memory (GB) :",
        round(torch.cuda.get_device_properties(0).total_memory / 1024**3, 2),
    )
print("=======================\n")


# -----------------------------
# Model loader with full prints
# -----------------------------
def load_model(kind: str):
    print(f"\n===== LOADING MODEL ({kind.upper()}) =====")

    if device == "cuda":
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()
        print("CUDA cache cleared")

    if kind == "base":
        model_path = "./models/Qwen3-TTS-12Hz-1.7B-Base"
    elif kind == "custom":
        model_path = "./models/Qwen3-TTS-12Hz-1.7B-CustomVoice"
    else:
        raise ValueError("kind must be 'base' or 'custom'")

    print("Model path      :", model_path)
    print("Torch dtype     :", dtype)

    model = Qwen3TTSModel.from_pretrained(
        model_path,
        dtype=dtype,
        device_map="auto",
    )

    # -----------------------------
    # Supported languages
    # -----------------------------
    try:
        langs = model.get_supported_languages()
        print("Supported languages:")
        for lang in langs:
            print(" -", lang)
    except Exception as e:
        print("Could not query supported languages:", e)

    # -----------------------------
    # Supported speakers (CustomVoice only)
    # -----------------------------
    try:
        speakers = model.get_supported_speakers()
        if speakers:
            print("Supported speakers:")
            for spk in speakers:
                print(" -", spk)
        else:
            print("Supported speakers: None (Base model)")
    except Exception:
        print("Supported speakers: Not available for this model")

    # -----------------------------
    # VRAM usage
    # -----------------------------
    if device == "cuda":
        allocated = torch.cuda.memory_allocated(0) / 1024**3
        reserved = torch.cuda.memory_reserved(0) / 1024**3
        peak = torch.cuda.max_memory_allocated(0) / 1024**3

        print(f"VRAM allocated  : {allocated:.2f} GB")
        print(f"VRAM reserved   : {reserved:.2f} GB")
        print(f"VRAM peak       : {peak:.2f} GB")

    print("Model loaded ✔")
    print("=================================\n")
    return model


# -----------------------------
# Model unload helper
# -----------------------------
def unload_model(model=None, name: str = ""):
    print(f"\n===== UNLOADING MODEL {name.upper()} =====")

    if model is not None:
        try:
            del model
            print("Model object deleted")
        except Exception as e:
            print("Model delete warning:", e)

    gc.collect()
    print("Python GC collected")

    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
        print("CUDA cache cleared")
        print(
            "VRAM after cleanup:",
            f"{torch.cuda.memory_allocated(0) / 1024**3:.2f} GB allocated",
        )

    print("====================================\n")


********
********
 
===== ENVIRONMENT =====
Device selected : cuda
Torch version   : 2.2.2+cu121
CUDA available  : True
CUDA version    : 12.1
GPU name        : Quadro P5000
GPU memory (GB) : 15.88



In [5]:
model = load_model("base")


===== LOADING MODEL (BASE) =====
CUDA cache cleared
Model path      : ./models/Qwen3-TTS-12Hz-1.7B-Base
Torch dtype     : torch.float16
Supported languages:
 - auto
 - chinese
 - english
 - french
 - german
 - italian
 - japanese
 - korean
 - portuguese
 - russian
 - spanish
Supported speakers: None (Base model)
VRAM allocated  : 1.37 GB
VRAM reserved   : 1.44 GB
VRAM peak       : 1.37 GB
Model loaded ✔



In [6]:
unload_model(model, name="base")
model = None


===== UNLOADING MODEL BASE =====
Model object deleted
Python GC collected
CUDA cache cleared
VRAM after cleanup: 1.37 GB allocated



In [5]:
# Load CustomVoice
model = load_model("custom")
speakers = model.get_supported_speakers()


===== LOADING MODEL (CUSTOM) =====
CUDA cache cleared
Model path      : ./models/Qwen3-TTS-12Hz-1.7B-CustomVoice
Torch dtype     : torch.float16
Supported languages:
 - auto
 - chinese
 - english
 - french
 - german
 - italian
 - japanese
 - korean
 - portuguese
 - russian
 - spanish
Supported speakers:
 - aiden
 - dylan
 - eric
 - ono_anna
 - ryan
 - serena
 - sohee
 - uncle_fu
 - vivian
VRAM allocated  : 2.81 GB
VRAM reserved   : 2.88 GB
VRAM peak       : 2.81 GB
Model loaded ✔



In [7]:
import torch
import soundfile as sf
from pathlib import Path


def run_generate_custom_voice(
    model,
    text,
    speaker,
    output_path,
    language="auto",
    instruct=None,
    generation_kwargs=None,
):
    """
    Explicit runner for Qwen3 CustomVoice TTS.

    This function does NOT inject defaults or modify user intent.
    All generation behavior is controlled by the caller.

    Parameters
    ----------
    model : Qwen3TTSModel
        Loaded CustomVoice model.
    text : str or list[str]
        Text(s) to synthesize.
    speaker : str or list[str]
        Speaker name(s), must match model.get_supported_speakers().
    output_path : str or Path
        Output WAV file (single) or directory (batch).
    language : str or list[str], default="auto"
        Language(s). Use "auto" for multilingual sentences.
    instruct : str or list[str] or None
        Optional style / emotion instruction(s).
    generation_kwargs : dict or None
        Extra kwargs passed directly to model.generate_custom_voice().
        (Nothing is added automatically.)
    """

    print("\n===== RUN generate_custom_voice =====")

    # -----------------------------
    # Determine mode
    # -----------------------------
    is_batch = isinstance(text, list)

    def ensure_list(value):
        return value if isinstance(value, list) else [value]

    text_list = ensure_list(text)
    speaker_list = ensure_list(speaker)
    language_list = ensure_list(language)
    instruct_list = ensure_list(instruct) if instruct is not None else None

    sample_count = len(text_list)

    # -----------------------------
    # Length validation
    # -----------------------------
    def validate_length(values, name):
        if values is not None and len(values) != sample_count:
            raise ValueError(
                f"{name} count mismatch: got {len(values)}, expected {sample_count}"
            )

    validate_length(speaker_list, "Speaker")
    validate_length(language_list, "Language")
    validate_length(instruct_list, "Instruct")

    # -----------------------------
    # Validate speaker & language
    # -----------------------------
    supported_speakers = set(model.get_supported_speakers())
    supported_languages = set(model.get_supported_languages())

    for spk in speaker_list:
        if spk not in supported_speakers:
            raise ValueError(
                f"Unsupported speaker '{spk}'. "
                f"Supported speakers: {sorted(supported_speakers)}"
            )

    for lang in language_list:
        if lang not in supported_languages:
            raise ValueError(
                f"Unsupported language '{lang}'. "
                f"Supported languages: {sorted(supported_languages)}"
            )

    # -----------------------------
    # Generation kwargs (explicit only)
    # -----------------------------
    if generation_kwargs is None:
        generation_kwargs = {}

    # -----------------------------
    # Output handling
    # -----------------------------
    output_path = Path(output_path)

    if is_batch:
        output_path.mkdir(parents=True, exist_ok=True)
    else:
        output_path.parent.mkdir(parents=True, exist_ok=True)

    # -----------------------------
    # User-facing summary
    # -----------------------------
    print("Batch mode        :", is_batch)
    print("Number of samples :", sample_count)
    print("Speaker(s)        :", speaker_list if is_batch else speaker_list[0])
    print("Language(s)       :", language_list if is_batch else language_list[0])
    print("Instruct          :", instruct_list if instruct_list else "None")
    print("Output path       :", output_path.resolve())
    print("Generation kwargs :", generation_kwargs)

    # -----------------------------
    # Generate audio
    # -----------------------------

    with torch.inference_mode():
        wavs, sample_rate = model.generate_custom_voice(
            text=text_list if is_batch else text_list[0],
            speaker=speaker_list if is_batch else speaker_list[0],
            language=language_list if is_batch else language_list[0],
            instruct=instruct_list if instruct_list is not None else None,
            **generation_kwargs,
        )

    # -----------------------------
    # Save results
    # -----------------------------
    if is_batch:
        for index, waveform in enumerate(wavs):
            output_file = output_path / f"custom_voice_{index}.wav"
            sf.write(output_file, waveform, sample_rate)
            print("Saved:", output_file)
    else:
        sf.write(output_path, wavs[0], sample_rate)
        print("Saved:", output_path)

    print("====================================\n")

In [8]:
text = (
    "Hello everyone, today I want to show you something very interesting. "
    "这个模型真的很厉害，boleh cakap English, Bahasa Melayu, "
    "dan 中文 together without any awkward pause. "
    "Kalau di Sarawak, orang akan cakap macam ni: "
    "'Aok, sik ada masalah bah, kitak rilek jak.' "
    "意思是说，everything is okay, no need to worry. "
    "You see ah, this kind of multilingual speech synthesis "
    "memang power, boleh campur bahasa ikut situasi. "
    "不是刻意的那种，而是自然流畅。"
)


run_generate_custom_voice(
    model=model,
    text=text,
    speaker="ryan",
    language="auto",
    output_path="outputs/multilang_ryan.wav",
)


===== RUN generate_custom_voice =====
Batch mode        : False
Number of samples : 1
Speaker(s)        : ryan
Language(s)       : auto
Instruct          : None
Output path       : /workspace/outputs/multilang_ryan.wav
Generation kwargs : {}


RuntimeError: CUDA error: CUBLAS_STATUS_INVALID_VALUE when calling `cublasGemmEx( handle, opa, opb, m, n, k, alpha_ptr, a, CUDA_R_16F, lda, b, CUDA_R_16F, ldb, beta_ptr, c, std::is_same_v<C_Dtype, float> ? CUDA_R_32F : CUDA_R_16F, ldc, compute_type, CUBLAS_GEMM_DEFAULT_TENSOR_OP)`

In [7]:
# Unload CustomVoice
unload_model(model, name="custom")
model = None


===== UNLOADING MODEL CUSTOM =====
Model object deleted
Python GC collected



### Example


In [1]:
import time
import gc
import torch
import soundfile as sf
from qwen_tts import Qwen3TTSModel


def select_runtime():
    if not torch.cuda.is_available():
        return {
            "device": "cpu",
            "dtype": torch.float32,
            "attn_impl": "eager",
        }

    major, minor = torch.cuda.get_device_capability(0)

    # Ampere+ (sm >= 80)
    if major >= 8:
        return {
            "device": "cuda",
            "dtype": torch.float16,  # bf16 optional if you prefer
            "attn_impl": "flash_attention_2",
        }

    # Turing / Pascal
    return {
        "device": "cuda",
        "dtype": torch.float16,
        "attn_impl": "eager",
    }


runtime = select_runtime()

print("===== RUNTIME CONFIG =====")
print("Device      :", runtime["device"])
print("DType       :", runtime["dtype"])
print("Attention   :", runtime["attn_impl"])
if torch.cuda.is_available():
    print("GPU         :", torch.cuda.get_device_name(0))
    print("Capability  :", torch.cuda.get_device_capability(0))
print("==========================\n")

MODEL_PATH = "./models/Qwen3-TTS-12Hz-1.7B-VoiceDesign"

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()

tts = Qwen3TTSModel.from_pretrained(
    MODEL_PATH,
    device_map="cuda:0",
    dtype=runtime["dtype"],
    attn_implementation=runtime["attn_impl"],
)

# tts.model.speech_tokenizer.model.decoder.pre_transformer.has_sliding_layers  = False
# for layer in tts.model.speech_tokenizer.model.decoder.pre_transformer.layers:
#     layer.attention_type = "full_attention"


********
********
 
===== RUNTIME CONFIG =====
Device      : cuda
DType       : torch.float16
Attention   : eager
GPU         : NVIDIA GeForce GTX 1660 Ti
Capability  : (7, 5)



In [ ]:
# -------- Single --------
if torch.cuda.is_available():
    torch.cuda.synchronize()
t0 = time.time()

text = (
    "哥哥，你回来啦？I’ve been waiting for you so long already，"
    "sampai人家都开始想你了。"
    "Really, you know? 每一分钟都觉得特别慢，"
    "macam masa tak mahu jalan。\n\n"
    "When I heard your footsteps just now，"
    "我心里一下子就亮起来了，terus rasa安心。"
    "You’re finally here, kan? Jangan pergi lagi lah，"
    "stay with me for a while。\n\n"
    "哥哥，抱我一下好不好？Just a little bit，kejap saja。"
    "我 promise 不吵你，只想靠着你，"
    "听你呼吸的声音。\n\n"
    "Everything feels okay now。"
    "有你在，什么都不怕了。"
)


wavs, sr = tts.generate_voice_design(
    text=text,
    language="auto",
    instruct=(
        "Soft, intimate and slightly playful young female voice. "
        "Tone is affectionate and clingy, with gentle pitch rises. "
        "Chinese lines should sound natural and warm, "
        "English calm and emotional, "
        "Malay casual and light, as if speaking softly to someone very close."
    ),
    max_new_tokens=2048,
)

if torch.cuda.is_available():
    torch.cuda.synchronize()
t1 = time.time()
print(f"[VoiceDesign Single] time: {t1 - t0:.3f}s")

sf.write("qwen3_tts_test_voice_design_single.wav", wavs[0], sr)

Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.
c:\Users\CodeAlpha\miniconda3\envs\tts\lib\site-packages\transformers\integrations\sdpa_attention.py:96: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [ ]:
# -------- Batch --------
texts = [
    # --- Sample 1: Chinese + English + Malay (clingy / cute) ---
    (
        "哥哥，你终于回来啦。I waited for you for so long already，"
        "sampai人家都有点委屈了。\n\n"
        "When you’re not around，"
        "时间走得特别慢，macam dunia停下来一样。\n\n"
        "现在你在这里了，"
        "我心就安了。哥哥，抱一下，好不好？"
    ),
    # --- Sample 2: English + Malay (soft emotional) ---
    (
        "I didn’t think you would come back so soon.\n"
        "But when I saw you，"
        "terus rasa lega。\n\n"
        "Just stay for a bit，okay？"
        "Jangan pergi lagi。"
    ),
    # --- Sample 3: Chinese-dominant with light English ---
    (
        "今天真的有点累了。\n"
        "But hearing your voice，"
        "突然就觉得没那么难受了。\n\n"
        "有你在，就够了。"
    ),
]
languages = ["auto", "auto", "auto"]
instructs = [
    # Sample 1 instruction
    (
        "Very cute, clingy, youthful female voice. "
        "Higher pitch with playful intonation. "
        "Chinese sounds soft and sweet, "
        "English emotional and gentle, "
        "Malay casual and affectionate."
    ),
    # Sample 2 instruction
    (
        "Soft, calm female voice. "
        "Emotionally warm and reassuring. "
        "Natural pacing, no exaggeration."
    ),
    # Sample 3 instruction
    (
        "Gentle, comforting female voice. "
        "Slightly tired but affectionate tone. "
        "Warm and intimate delivery."
    ),
]

if torch.cuda.is_available():
    torch.cuda.synchronize()
t0 = time.time()

wavs, sr = tts.generate_voice_design(
    text=texts,
    language=languages,
    instruct=instructs,
    max_new_tokens=2048,
)

if torch.cuda.is_available():
    torch.cuda.synchronize()
t1 = time.time()
print(f"[VoiceDesign Batch] time: {t1 - t0:.3f}s")

for i, w in enumerate(wavs):
    sf.write(f"qwen3_tts_test_voice_design_batch_{i}.wav", w, sr)

In [ ]:
# Cleanup
del tts
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()